In [5]:
import sqlite3

def main():
    # Path to your athome_listings.db database file
    db_path = "athome_listings.db"

    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # SQL query:
    #  - Reformat the createdAt and soldAt dates.
    #  - Only consider listings created from 2010 onwards which have a soldAt date.
    #  - Only use listings where the soldAt date is later than or equal to createdAt.
    #  - Calculate:
    #      · The number of transactions per group
    #      · The average number of days a listing was open.
    #      · The average m²/€ ratio (only when price > 0 and "characteristic.surface"
    #        is not NULL and greater than 0).
    query = """
    SELECT 
      CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) AS year,
      [transaction],
      COUNT(*) AS transaction_count,
      AVG(
        JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
        JULIANDAY(
          SUBSTR(createdAt, 1, 4) || '-' ||
          SUBSTR(createdAt, 5, 2) || '-' ||
          SUBSTR(createdAt, 7, 2) || ' ' ||
          SUBSTR(createdAt, 10, 2) || ':' ||
          SUBSTR(createdAt, 12, 2) || ':' ||
          SUBSTR(createdAt, 14, 2)
        )
      ) AS avg_open_days,
      AVG(
         CASE 
           WHEN price > 0 AND "characteristic.surface" IS NOT NULL
                AND "characteristic.surface" > 0
           THEN "characteristic.surface" / price
         END
      ) AS avg_m2_per_euro
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) >= 2010
      AND soldAt IS NOT NULL
      AND [transaction] IN ('rent', 'buy')
      AND (
        JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
        JULIANDAY(
          SUBSTR(createdAt, 1, 4) || '-' ||
          SUBSTR(createdAt, 5, 2) || '-' ||
          SUBSTR(createdAt, 7, 2) || ' ' ||
          SUBSTR(createdAt, 10, 2) || ':' ||
          SUBSTR(createdAt, 12, 2) || ':' ||
          SUBSTR(createdAt, 14, 2)
        )
      ) >= 0
    GROUP BY year, [transaction]
    ORDER BY year, [transaction];
    """

    try:
        # Execute the query and fetch the results
        cursor.execute(query)
        results = cursor.fetchall()

        # Print a header
        header = (
            f"{'Year':<6} | {'Type':<8} | {'Count':<7} | "
            f"{'Avg Open Days':<14} | {'Avg m²/€':<10}"
        )
        print(header)
        print("-" * len(header))

        # Print each row of results
        for row in results:
            year, trans, cnt, avg_open_days, avg_m2_per_euro = row
            avg_days_str = f"{avg_open_days:.2f}" if avg_open_days is not None else "None"
            # Only show avg_m2_per_euro when available; otherwise "None"
            avg_m2_str = f"{avg_m2_per_euro:.4f}" if avg_m2_per_euro is not None else "None"
            print(f"{year:<6} | {trans:<8} | {cnt:<7} | {avg_days_str:<14} | {avg_m2_str:<10}")

    except sqlite3.OperationalError as e:
        print("SQLite error:", e)
    finally:
        # Close the database connection
        conn.close()

if __name__ == "__main__":
    main()


Year   | Type     | Count   | Avg Open Days  | Avg m²/€  
---------------------------------------------------------
2010   | buy      | 9       | 4677.00        | 0.0003    
2011   | buy      | 295     | 4565.02        | 0.0007    
2012   | buy      | 316     | 4298.51        | 0.0004    
2012   | rent     | 1       | 4194.05        | 0.0380    
2013   | buy      | 107     | 3871.95        | 0.0003    
2013   | rent     | 1       | 4008.02        | 0.0152    
2014   | buy      | 68      | 3491.28        | 0.0004    
2015   | buy      | 98      | 2901.60        | 0.0004    
2016   | buy      | 80      | 2623.54        | 0.0004    
2016   | rent     | 1       | 2699.06        | 0.0180    
2017   | buy      | 57      | 1825.24        | 0.0003    
2018   | buy      | 114     | 1136.42        | 7.6517    
2018   | rent     | 1       | 1976.77        | 0.0225    
2019   | buy      | 207     | 669.82         | 3.2191    
2019   | rent     | 5       | 1800.36        | 0.0530    
2020   | buy  

In [6]:
import sqlite3

def debug_2025_rent():
    db_path = "athome_listings.db"  # Adjust if necessary
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Debug query: select the id, price, surface, and ratio for listings in 2025 with rent
    query_debug = """
    SELECT 
      id,
      price,
      "characteristic.surface" AS surface,
      ("characteristic.surface") / price AS ratio,
      SUBSTR(createdAt, 1, 4) AS year,
      [transaction],
      soldAt
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) = 2025
      AND [transaction] = 'rent'
      AND soldAt IS NOT NULL
      AND price > 0
      AND "characteristic.surface" IS NOT NULL
      AND "characteristic.surface" > 0
      AND (
            JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' '))
            - JULIANDAY(
                SUBSTR(createdAt, 1, 4) || '-' ||
                SUBSTR(createdAt, 5, 2) || '-' ||
                SUBSTR(createdAt, 7, 2) || ' ' ||
                SUBSTR(createdAt, 10, 2) || ':' ||
                SUBSTR(createdAt, 12, 2) || ':' ||
                SUBSTR(createdAt, 14, 2)
              )
          ) >= 0
    ORDER BY id;
    """
    
    try:
        cursor.execute(query_debug)
        rows = cursor.fetchall()
        
        print("Debug Data for 2025 rent listings:")
        print("ID   | Price     | Surface (m²) | Ratio (m²/€)")
        print("-" * 50)
        
        for row in rows:
            listing_id, price, surface, ratio, year, trans, soldAt = row
            print(f"{listing_id:<4} | {price:<9} | {surface:<12} | {ratio:<10.5f}")
        
        # Optionally, get aggregated average for comparison
        query_avg = """
        SELECT AVG( ("characteristic.surface") / price )
        FROM listings
        WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) = 2025
          AND [transaction] = 'rent'
          AND soldAt IS NOT NULL
          AND price > 0
          AND "characteristic.surface" IS NOT NULL
          AND "characteristic.surface" > 0
          AND (
                JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' '))
                - JULIANDAY(
                    SUBSTR(createdAt, 1, 4) || '-' ||
                    SUBSTR(createdAt, 5, 2) || '-' ||
                    SUBSTR(createdAt, 7, 2) || ' ' ||
                    SUBSTR(createdAt, 10, 2) || ':' ||
                    SUBSTR(createdAt, 12, 2) || ':' ||
                    SUBSTR(createdAt, 14, 2)
                  )
              ) >= 0;
        """
        cursor.execute(query_avg)
        avg_value = cursor.fetchone()[0]
        print("\nAggregated average m²/€ for 2025 rent:", 
              f"{avg_value:.5f}" if avg_value is not None else "None")
        
    except sqlite3.OperationalError as e:
        print("SQLite error:", e)
    finally:
        conn.close()

if __name__ == "__main__":
    debug_2025_rent()


Debug Data for 2025 rent listings:
ID   | Price     | Surface (m²) | Ratio (m²/€)
--------------------------------------------------
8435530 | 1050.0    | 45.0         | 0.04286   
8435704 | 1000.0    | 22.0         | 0.02200   
8435704 | 1000.0    | 22.0         | 0.02200   
8435736 | 1450.0    | 38.0         | 0.02621   
8435880 | 1800.0    | 65.0         | 0.03611   
8435895 | 2000.0    | 90.0         | 0.04500   
8435895 | 2000.0    | 90.0         | 0.04500   
8436506 | 1700.0    | 70.0         | 0.04118   
8436546 | 1700.0    | 85.0         | 0.05000   
8436831 | 1850.0    | 56.0         | 0.03027   
8437497 | 850.0     | 11.0         | 0.01294   
8440552 | 1200.0    | 90.0         | 0.07500   
8441431 | 2300.0    | 104.0        | 0.04522   
8441819 | 246.0     | 7.0          | 0.02846   
8441820 | 1000.0    | 22.0         | 0.02200   
8441820 | 1000.0    | 22.0         | 0.02200   
8441825 | 397.0     | 11.0         | 0.02771   
8442504 | 1800.0    | 60.0         | 0.03333   
844

In [7]:
import sqlite3
import numpy as np

def debug_2025_rent_ratio():
    db_path = "athome_listings.db"  # Adjust the filename/path if needed
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # SQL query to fetch listings for 2025 with transaction 'rent'
    # The ratio is calculated as price divided by "characteristic.surface" (€/m²)
    # The posted conditions exclude rows with invalid data.
    query = """
    SELECT 
      id,
      price,
      "characteristic.surface" AS surface,
      price / "characteristic.surface" AS ratio, 
      SUBSTR(createdAt, 1, 4) AS year,
      [transaction],
      soldAt
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) = 2025
      AND [transaction] = 'rent'
      AND soldAt IS NOT NULL
      AND price > 0
      AND "characteristic.surface" IS NOT NULL
      AND "characteristic.surface" > 0
      AND (
        JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
        JULIANDAY(
          SUBSTR(createdAt, 1, 4) || '-' ||
          SUBSTR(createdAt, 5, 2) || '-' ||
          SUBSTR(createdAt, 7, 2) || ' ' ||
          SUBSTR(createdAt, 10, 2) || ':' ||
          SUBSTR(createdAt, 12, 2) || ':' ||
          SUBSTR(createdAt, 14, 2)
        )
      ) >= 0
    ORDER BY id;
    """

    try:
        cursor.execute(query)
        rows = cursor.fetchall()

        print("Debug Data for 2025 rent listings (€/m² ratio):")
        print("ID   | Price (€)  | Surface (m²) | Ratio (€/m²)")
        print("-" * 60)

        ratios = []  # Store individual ratio values

        for row in rows:
            listing_id, price, surface, ratio, year, trans, soldAt = row
            print(f"{listing_id:<4} | {price:<10.2f} | {surface:<12.2f} | {ratio:<12.4f}")
            ratios.append(ratio)

        if not ratios:
            print("No listings found matching criteria.")
        else:
            # Convert ratios list to a NumPy array for statistical calculations.
            ratios_array = np.array(ratios)
            average = ratios_array.mean()
            median = np.median(ratios_array)
            q1 = np.percentile(ratios_array, 25)
            q3 = np.percentile(ratios_array, 75)

            print("\nAggregated Statistics for 2025 rent (€/m²):")
            print(f"Count: {len(ratios)}")
            print(f"Average: {average:.4f}")
            print(f"Median:  {median:.4f}")
            print(f"Q1 (25th percentile): {q1:.4f}")
            print(f"Q3 (75th percentile): {q3:.4f}")

    except sqlite3.OperationalError as e:
        print("SQLite error:", e)
    finally:
        conn.close()

if __name__ == "__main__":
    debug_2025_rent_ratio()


Debug Data for 2025 rent listings (€/m² ratio):
ID   | Price (€)  | Surface (m²) | Ratio (€/m²)
------------------------------------------------------------
8435530 | 1050.00    | 45.00        | 23.3333     
8435704 | 1000.00    | 22.00        | 45.4545     
8435704 | 1000.00    | 22.00        | 45.4545     
8435736 | 1450.00    | 38.00        | 38.1579     
8435880 | 1800.00    | 65.00        | 27.6923     
8435895 | 2000.00    | 90.00        | 22.2222     
8435895 | 2000.00    | 90.00        | 22.2222     
8436506 | 1700.00    | 70.00        | 24.2857     
8436546 | 1700.00    | 85.00        | 20.0000     
8436831 | 1850.00    | 56.00        | 33.0357     
8437497 | 850.00     | 11.00        | 77.2727     
8440552 | 1200.00    | 90.00        | 13.3333     
8441431 | 2300.00    | 104.00       | 22.1154     
8441819 | 246.00     | 7.00         | 35.1429     
8441820 | 1000.00    | 22.00        | 45.4545     
8441820 | 1000.00    | 22.00        | 45.4545     
8441825 | 397.00     | 11.0

In [8]:
import sqlite3
import numpy as np

def debug_all_years_ratio():
    db_path = "athome_listings.db"  # Adjust your database path if needed
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # SQL query retrieves all listings from 2010 onward where:
    # - soldAt is available and the listing was sold after (or at) the created time
    # - price (> 0) and "characteristic.surface" (> 0) are valid
    # - [transaction] is rent or buy
    # The €/m² ratio is calculated as price divided by surface.
    query = """
    SELECT 
        id,
        price,
        "characteristic.surface" AS surface,
        price / "characteristic.surface" AS ratio,
        CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) AS year,
        [transaction],
        soldAt
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) >= 2010
      AND soldAt IS NOT NULL
      AND [transaction] IN ('rent', 'buy')
      AND price > 0
      AND "characteristic.surface" IS NOT NULL
      AND "characteristic.surface" > 0
      AND (
           JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
           JULIANDAY(
             SUBSTR(createdAt, 1, 4) || '-' ||
             SUBSTR(createdAt, 5, 2) || '-' ||
             SUBSTR(createdAt, 7, 2) || ' ' ||
             SUBSTR(createdAt, 10, 2) || ':' ||
             SUBSTR(createdAt, 12, 2) || ':' ||
             SUBSTR(createdAt, 14, 2)
           )
      ) >= 0
    ORDER BY year, [transaction], id;
    """

    try:
        cursor.execute(query)
        rows = cursor.fetchall()

        # Group rows by (year, transaction)
        data_by_group = {}
        for row in rows:
            listing_id, price, surface, ratio, year, trans, soldAt = row
            key = (year, trans)
            if key not in data_by_group:
                data_by_group[key] = []
            data_by_group[key].append({
                "id": listing_id,
                "price": price,
                "surface": surface,
                "ratio": ratio,
                "soldAt": soldAt
            })

        # Print aggregated statistics for each group.
        print("Aggregated €/m² statistics for all years (€/m² = price / surface):")
        header = (
            f"{'Year':>4} | {'Transaction':>10} | {'Count':>6} | "
            f"{'Avg (€/m²)':>12} | {'Median':>12} | {'Q1':>12} | {'Q3':>12}"
        )
        print(header)
        print("-" * len(header))

        # Iterate over groups in sorted order and calculate aggregates.
        for key in sorted(data_by_group.keys()):
            year, trans = key
            group_data = data_by_group[key]
            ratios = np.array([entry["ratio"] for entry in group_data])
            count = len(ratios)
            average = ratios.mean()
            median = np.median(ratios)
            q1 = np.percentile(ratios, 25)
            q3 = np.percentile(ratios, 75)
            print(
                f"{year:>4} | {trans:>10} | {count:6d} | "
                f"{average:12.4f} | {median:12.4f} | {q1:12.4f} | {q3:12.4f}"
            )

    except sqlite3.OperationalError as e:
        print("SQLite error:", e)
    finally:
        conn.close()

if __name__ == "__main__":
    debug_all_years_ratio()


Aggregated €/m² statistics for all years (€/m² = price / surface):
Year | Transaction |  Count |   Avg (€/m²) |       Median |           Q1 |           Q3
---------------------------------------------------------------------------------------
2010 |        buy |      5 |    3286.8653 |    3041.4414 |    2998.4615 |    3255.4847
2011 |        buy |    246 |    2548.0447 |    2586.5385 |    2241.6667 |    2864.7059
2012 |        buy |    261 |    2884.1818 |    2700.0000 |    2434.7826 |    3466.6667
2012 |       rent |      1 |      26.3158 |      26.3158 |      26.3158 |      26.3158
2013 |        buy |     80 |    3358.1037 |    3247.7810 |    2814.9962 |    3884.2647
2013 |       rent |      1 |      65.8333 |      65.8333 |      65.8333 |      65.8333
2014 |        buy |     63 |    3387.6311 |    3040.0000 |    2519.7917 |    3955.1163
2015 |        buy |     78 |    3390.6797 |    2904.3445 |    2712.6431 |    3684.2105
2016 |        buy |     66 |    3374.6313 |    3052.5266 |   

In [10]:
import sqlite3
import numpy as np

def debug_all_years_by_country_ratio():
    db_path = "athome_listings.db"  # Adjust your DB path if needed
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # SQL query retrieves all listings from 2010 onward that:
    # - Have a soldAt date (and soldAt is after or equal to createdAt)
    # - Have a valid price (> 0) and a valid "characteristic.surface" (> 0)
    # - Are either rent or buy transactions.
    # The €/m² ratio is computed as price / "characteristic.surface"
    query = """
    SELECT 
        id,
        price,
        "characteristic.surface" AS surface,
        price / "characteristic.surface" AS ratio,
        CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) AS year,
        [transaction],
        "address.country" AS country,
        soldAt
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) >= 2010
      AND soldAt IS NOT NULL
      AND [transaction] IN ('rent', 'buy')
      AND price > 0
      AND "characteristic.surface" IS NOT NULL
      AND "characteristic.surface" > 0
      AND (
          JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
          JULIANDAY(
            SUBSTR(createdAt, 1, 4) || '-' ||
            SUBSTR(createdAt, 5, 2) || '-' ||
            SUBSTR(createdAt, 7, 2) || ' ' ||
            SUBSTR(createdAt, 10, 2) || ':' ||
            SUBSTR(createdAt, 12, 2) || ':' ||
            SUBSTR(createdAt, 14, 2)
          )
      ) >= 0
    ORDER BY year, [transaction], country, id;
    """

    try:
        cursor.execute(query)
        rows = cursor.fetchall()

        # Group data by (year, transaction, country)
        groups = {}
        for row in rows:
            listing_id, price, surface, ratio, year, trans, country, soldAt = row
            key = (year, trans, country)
            groups.setdefault(key, []).append(ratio)

        # Print aggregated statistics for each group.
        header = (
            f"{'Year':>4} | {'Country':>10} | {'Transaction':>10} | {'Count':>6} | "
            f"{'Average':>10} | {'Median':>10} | {'Q1':>10} | {'Q3':>10}"
        )
        print("Aggregated €/m² Ratio Statistics (€/m² = price / surface) by Country:")
        print(header)
        print("-" * len(header))

        # Sort keys while substituting a default value for None.
        sorted_keys = sorted(
            groups.keys(), key=lambda k: (k[0], k[1], k[2] if k[2] is not None else "")
        )
        for key in sorted_keys:
            year, trans, country = key
            ratios = np.array(groups[key])
            count = len(ratios)
            avg = ratios.mean()
            median = np.median(ratios)
            q1 = np.percentile(ratios, 25)
            q3 = np.percentile(ratios, 75)
            country_str = country if country is not None else "Unknown"
            print(
                f"{year:>4} | {country_str:>10} | {trans:>10} | {count:6d} | "
                f"{avg:10.4f} | {median:10.4f} | {q1:10.4f} | {q3:10.4f}"
            )

    except sqlite3.OperationalError as e:
        print("SQLite error:", e)
    finally:
        conn.close()

if __name__ == "__main__":
    debug_all_years_by_country_ratio()


Aggregated €/m² Ratio Statistics (€/m² = price / surface) by Country:
Year |    Country | Transaction |  Count |    Average |     Median |         Q1 |         Q3
--------------------------------------------------------------------------------------------
2010 | Luxembourg |        buy |      5 |  3286.8653 |  3041.4414 |  2998.4615 |  3255.4847
2011 |    Belgium |        buy |     12 |  1086.6735 |   934.4262 |   893.4426 |  1244.7981
2011 | Luxembourg |        buy |    234 |  2622.9868 |  2673.0769 |  2266.9960 |  2864.7059
2012 |    Belgium |        buy |      6 |  1677.8946 |  1613.0464 |   934.4262 |  2441.6667
2012 |    Germany |        buy |      1 |  1500.0000 |  1500.0000 |  1500.0000 |  1500.0000
2012 | Luxembourg |        buy |    254 |  2918.1263 |  2718.1818 |  2448.2759 |  3486.8421
2012 | Luxembourg |       rent |      1 |    26.3158 |    26.3158 |    26.3158 |    26.3158
2013 |    Belgium |        buy |      1 |  2339.1304 |  2339.1304 |  2339.1304 |  2339.1304
2013 |  

In [14]:
import sqlite3
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def fetch_aggregated_sold_data(db_path="athome_listings.db"):
    """
    Retrieves and aggregates data for listings that are sold.
    For each listing the €/m² ratio is computed as price / "characteristic.surface"
    and open_days is calculated as the difference between soldAt and createdAt.
    Data is grouped by (year, transaction, country). Only groups with 5+ data points are kept.
    The aggregated statistics are computed using the median (with quartiles).
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    query = """
    SELECT 
        id,
        price,
        "characteristic.surface" AS surface,
        price / "characteristic.surface" AS ratio,
        CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) AS year,
        [transaction],
        "address.country" AS country,
        soldAt,
        (
          JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
          JULIANDAY(
            SUBSTR(createdAt, 1, 4) || '-' ||
            SUBSTR(createdAt, 5, 2) || '-' ||
            SUBSTR(createdAt, 7, 2) || ' ' ||
            SUBSTR(createdAt, 10, 2) || ':' ||
            SUBSTR(createdAt, 12, 2) || ':' ||
            SUBSTR(createdAt, 14, 2)
          )
        ) AS open_days
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) >= 2010
      AND soldAt IS NOT NULL
      AND [transaction] IN ('rent', 'buy')
      AND price > 0
      AND "characteristic.surface" IS NOT NULL
      AND "characteristic.surface" > 0
      AND (
          JULIANDAY(REPLACE(SUBSTR(soldAt, 1, 19), 'T', ' ')) -
          JULIANDAY(
            SUBSTR(createdAt, 1, 4) || '-' ||
            SUBSTR(createdAt, 5, 2) || '-' ||
            SUBSTR(createdAt, 7, 2) || ' ' ||
            SUBSTR(createdAt, 10, 2) || ':' ||
            SUBSTR(createdAt, 12, 2) || ':' ||
            SUBSTR(createdAt, 14, 2)
          )
      ) >= 0
    ORDER BY year, [transaction], country, id;
    """

    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()

    # Group by (year, transaction, country)
    groups = {}
    for row in rows:
        # Row structure:
        # (id, price, surface, ratio, year, transaction, country, soldAt, open_days)
        _, _, _, ratio, year, tran, country, _, open_days = row
        key = (year, tran, country)
        groups.setdefault(key, {"ratios": [], "open_days": []})
        groups[key]["ratios"].append(ratio)
        groups[key]["open_days"].append(open_days)

    data = []
    for key, stats in groups.items():
        count = len(stats["ratios"])
        if count < 5:
            continue  # Skip groups with fewer than 5 listings.
        year, tran, country = key
        arr_ratio = np.array(stats["ratios"])
        arr_days = np.array(stats["open_days"])
        data.append({
            "year": year,
            "transaction": tran,
            "country": country if country is not None else "Unknown",
            "count": count,
            "median_ratio": np.median(arr_ratio),
            "q1_ratio": np.percentile(arr_ratio, 25),
            "q3_ratio": np.percentile(arr_ratio, 75),
            "median_open_days": np.median(arr_days),
            "q1_open_days": np.percentile(arr_days, 25),
            "q3_open_days": np.percentile(arr_days, 75)
        })

    return pd.DataFrame(data)

def fetch_aggregated_unsold_data(db_path="athome_listings.db"):
    """
    Retrieves and aggregates data for listings that are not sold.
    For unsold listings, open_days is computed as the difference between now and createdAt.
    Data is grouped by (year, transaction, country) and only groups with 5+ data points are kept.
    Aggregated statistics (median and quartiles) for ratio and open_days are computed.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    query = """
    SELECT 
        id,
        price,
        "characteristic.surface" AS surface,
        price / "characteristic.surface" AS ratio,
        CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) AS year,
        [transaction],
        "address.country" AS country,
        createdAt,
        (
          JULIANDAY('now') -
          JULIANDAY(
            SUBSTR(createdAt, 1, 4) || '-' ||
            SUBSTR(createdAt, 5, 2) || '-' ||
            SUBSTR(createdAt, 7, 2) || ' ' ||
            SUBSTR(createdAt, 10, 2) || ':' ||
            SUBSTR(createdAt, 12, 2) || ':' ||
            SUBSTR(createdAt, 14, 2)
          )
        ) AS open_days
    FROM listings
    WHERE CAST(SUBSTR(createdAt, 1, 4) AS INTEGER) >= 2010
      AND soldAt IS NULL
      AND [transaction] IN ('rent', 'buy')
      AND price > 0
      AND "characteristic.surface" IS NOT NULL
      AND "characteristic.surface" > 0
    ORDER BY year, [transaction], country, id;
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()

    # Group by (year, transaction, country)
    groups = {}
    for row in rows:
        # Row structure:
        # (id, price, surface, ratio, year, transaction, country, createdAt, open_days)
        _, _, _, ratio, year, tran, country, _, open_days = row
        key = (year, tran, country)
        groups.setdefault(key, {"ratios": [], "open_days": []})
        groups[key]["ratios"].append(ratio)
        groups[key]["open_days"].append(open_days)

    data = []
    for key, stats in groups.items():
        count = len(stats["ratios"])
        if count < 5:
            continue
        year, tran, country = key
        arr_ratio = np.array(stats["ratios"])
        arr_days = np.array(stats["open_days"])
        data.append({
            "year": year,
            "transaction": tran,
            "country": country if country is not None else "Unknown",
            "count": count,
            "median_ratio": np.median(arr_ratio),
            "q1_ratio": np.percentile(arr_ratio, 25),
            "q3_ratio": np.percentile(arr_ratio, 75),
            "median_open_days": np.median(arr_days),
            "q1_open_days": np.percentile(arr_days, 25),
            "q3_open_days": np.percentile(arr_days, 75)
        })

    return pd.DataFrame(data)

def generate_html_report(sold_df, unsold_df, output_html="report.html"):
    """
    Generates an interactive HTML report with two main sections:
     1. Sold Listings Analysis.
     2. Unsold Listings Analysis.

    For each section an interactive table and bar charts (using median values) are produced.
    """
    html_parts = []
    html_parts.append("<html><head><title>Aggregated Data Report</title></head><body>")
    html_parts.append("<h1>Aggregated Statistics Report</h1>")

    # ------------------------------
    # Section 1: Sold Listings Analysis
    # ------------------------------
    html_parts.append("<h2>Section 1: Sold Listings Analysis</h2>")
    if sold_df.empty:
        html_parts.append("<p>No sold listings groups with at least 5 data points found.</p>")
    else:
        sold_df_sorted = sold_df.sort_values(by=["year", "country", "transaction"])
        # Create a table for sold listings.
        table_sold = go.Figure(data=[go.Table(
            header=dict(
                values=list(sold_df_sorted.columns),
                fill_color="paleturquoise",
                align="left"
            ),
            cells=dict(
                values=[sold_df_sorted[col] for col in sold_df_sorted.columns],
                fill_color="lavender",
                align="left"
            )
        )])
        table_sold.update_layout(title="Sold Listings Statistics")
        html_parts.append(table_sold.to_html(full_html=False, include_plotlyjs="cdn"))

        # Bar chart for median €/m² ratio for sold listings.
        bar_sold_ratio = px.bar(
            sold_df,
            x="year",
            y="median_ratio",
            color="transaction",
            facet_col="country",
            facet_col_wrap=3,
            title="Median €/m² Ratio (Sold Listings)",
            labels={"median_ratio": "Median (€/m²)", "year": "Year"}
        )
        bar_sold_ratio.update_layout(showlegend=True)
        html_parts.append("<h3>Median €/m² Ratio (Sold Listings)</h3>")
        html_parts.append(bar_sold_ratio.to_html(full_html=False, include_plotlyjs=False))

        # Bar chart for median open days for sold listings.
        bar_sold_open = px.bar(
            sold_df,
            x="year",
            y="median_open_days",
            color="transaction",
            facet_col="country",
            facet_col_wrap=3,
            title="Median Time to Sell (Open Days) (Sold Listings)",
            labels={"median_open_days": "Median Open Days", "year": "Year"}
        )
        bar_sold_open.update_layout(showlegend=True)
        html_parts.append("<h3>Median Time to Sell (Open Days) (Sold Listings)</h3>")
        html_parts.append(bar_sold_open.to_html(full_html=False, include_plotlyjs=False))

    # ------------------------------
    # Section 2: Unsold Listings Analysis
    # ------------------------------
    html_parts.append("<h2>Section 2: Unsold Listings Analysis</h2>")
    if unsold_df.empty:
        html_parts.append("<p>No unsold listings groups with at least 5 data points found.</p>")
    else:
        unsold_df_sorted = unsold_df.sort_values(by=["year", "country", "transaction"])
        # Create a table for unsold listings.
        table_unsold = go.Figure(data=[go.Table(
            header=dict(
                values=list(unsold_df_sorted.columns),
                fill_color="paleturquoise",
                align="left"
            ),
            cells=dict(
                values=[unsold_df_sorted[col] for col in unsold_df_sorted.columns],
                fill_color="lavender",
                align="left"
            )
        )])
        table_unsold.update_layout(title="Unsold Listings Statistics")
        html_parts.append(table_unsold.to_html(full_html=False, include_plotlyjs="cdn"))

        # Bar chart for median €/m² ratio for unsold listings.
        bar_unsold_ratio = px.bar(
            unsold_df,
            x="year",
            y="median_ratio",
            color="transaction",
            facet_col="country",
            facet_col_wrap=3,
            title="Median €/m² Ratio (Unsold Listings)",
            labels={"median_ratio": "Median (€/m²)", "year": "Year"}
        )
        bar_unsold_ratio.update_layout(showlegend=True)
        html_parts.append("<h3>Median €/m² Ratio (Unsold Listings)</h3>")
        html_parts.append(bar_unsold_ratio.to_html(full_html=False, include_plotlyjs=False))

        # Bar chart for median open days for unsold listings.
        bar_unsold_open = px.bar(
            unsold_df,
            x="year",
            y="median_open_days",
            color="transaction",
            facet_col="country",
            facet_col_wrap=3,
            title="Median Time on Market (Open Days) (Unsold Listings)",
            labels={"median_open_days": "Median Open Days", "year": "Year"}
        )
        bar_unsold_open.update_layout(showlegend=True)
        html_parts.append("<h3>Median Time on Market (Open Days) (Unsold Listings)</h3>")
        html_parts.append(bar_unsold_open.to_html(full_html=False, include_plotlyjs=False))

    html_parts.append("</body></html>")

    with open(output_html, "w", encoding="utf-8") as f:
        f.write("\n".join(html_parts))

    print(f"Report saved to {output_html}")

def main():
    sold_df = fetch_aggregated_sold_data()
    unsold_df = fetch_aggregated_unsold_data()
    generate_html_report(sold_df, unsold_df)

if __name__ == "__main__":
    main()


Report saved to report.html
